In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from tqdm import tqdm
from sklearn.neighbors import NearestNeighbors
import numpy as np
import random
import pickle 

def scrape_goodreads_ratings(user_id, max_pages=10):
    """
    Scrape a user's star ratings from Goodreads.
    
    Args:
    - user_id (str): Goodreads user ID or profile suffix.
    - max_pages (int): Maximum number of pages to scrape (each page contains ~30 books).
    
    Returns:
    - pd.DataFrame: A DataFrame containing book titles and ratings.
    """
    base_url = f"https://www.goodreads.com/review/list/{user_id}?shelf=read"
    headers = {"User-Agent": "Mozilla/5.0"}
    books = []

    for page in range(1, max_pages + 1):
        url = f"{base_url}&page={page}"
        response = requests.get(url, headers=headers)
        
        if response.status_code != 200:
            print(f"Failed to fetch page {page}. Status code: {response.status_code}")
            break

        soup = BeautifulSoup(response.content, "html.parser")
        
        # Find all book entries in the table
        rows = soup.find_all("tr", class_="bookalike review")
        if not rows:
            print("No more data found.")
            break

        for row in rows:
            try:
                title = row.find("td", class_="field title").a.text.strip()
                rating_element = row.find("td", class_="field rating")
                rating = rating_element.find("span", class_="staticStars").get("title", "No rating")
                stars = map_rating(rating)
                books.append({"Title": title, "Rating": stars, "User_id": user_id})
#                 print(title, rating, stars)
            except AttributeError:
                # Handle rows with missing data
                continue

        print(f"Page {page} scraped successfully.")
        time.sleep(random.uniform(1, 5))  # Be kind to the server and avoid being blocked

    # Return data as a pandas DataFrame
    return pd.DataFrame(books)



In [2]:
def map_rating(phrase):
    rating_map = {
        "liked it": 3,
        "really liked it": 4,
        "it was ok": 2, 
        "it was amazing": 5, 
        "did not like it": 1,
    }
    
    return rating_map.get(phrase, "Invalid rating")  # Default to "Invalid rating" if the phrase isn't in the dictionary


In [3]:
# if __name__ == "__main__":
# #     user_id = "6688207"  # Replace with the Goodreads user ID or profile suffix
# #     for user_id in tqdm(['30181442', '75009563', '11345366', '110912303', '113964939', '11215896', '53701594', '4622890', '93628736', '176180116']):
# #     for user_id in tqdm(['2974095', '4622890', '28953843', '16174645', '4159922', '4125660', '54886546', '16912659', '260116', '4685500', '21865425']):
# #     for user_id in tqdm(['53701594', '27709782', '7566229', '16652861', '30817744', '56259255', '4125660', '60964126', 
# #                          '176167767', '28510930', '1029975', '131020767', '28862120', '88713906', '160141433', '41097916', 
# #                          '20809863', '69519261', '24017481', '7376365', '75941333', '13571407', '106618742', '17792052',
# #                          '3534528', '130656897', '7474475', '4125412', '6336365', '6026811', '3438047']):
#     for user_id in ['169695556']:
#         print("User_id = ", user_id)
#         max_pages = 30  # Adjust based on expected data
#         ratings_data = scrape_goodreads_ratings(user_id, max_pages)

#         if not ratings_data.empty:
# #             print(ratings_data.head())
# #             ratings_data.to_csv("goodreads_ratings.csv", index=False)
#             ratings_data.to_csv('goodreads_ratings.csv', mode='a', header=False, index=False)
#             print("Data saved to goodreads_ratings.csv.")
#         else:
#             print("No data retrieved.")


In [4]:
df = pd.read_csv('goodreads_ratings_series.csv')
print(df.shape)
df = df.drop_duplicates()
# Print the entire DataFrame
print(df)

duplicate_count = df['Title'].duplicated().sum()
print("Number of books with at least two people rating it:", duplicate_count)
duplicate_counts_per_value = df['Title'].value_counts()
print(duplicate_counts_per_value)
print("Number of unique books: ", df['Title'].nunique())
num_users = df['User_id'].nunique()
user_ids = list(df['User_id'].unique())
print("number of users is: ", num_users)
print("user_ids = ", user_ids)

(209600, 6)
                                                    Title Rating    User_id  \
0                                       I Am Watching You      3  169695558   
1       Three to Get Deadly\n        (Stephanie Plum, #3)      3  169695558   
2       Before the Coffee Gets Cold\n        (Before t...      4  169695558   
3       Dark Sacred Night\n        (Renée Ballard, #2;...      4  169695558   
4         Two for the Dough\n        (Stephanie Plum, #2)      4  169695558   
...                                                   ...    ...        ...   
209595                            The Old Man and the Sea      5    9497971   
209596  The Hitchhikerâ€™s Guide to the Galaxy\n      ...      5    9497971   
209597                                Slaughterhouse-Five      5    9497971   
209598                              To Kill a Mockingbird      5    9497971   
209599  Ben BuradayÄ±m: OÄŸuz Atay'Ä±n Biyografik ve K...      4    9497972   

        Series  First  Suggest  
0     

In [5]:
# # Get a list of top titles in order
# top_titles = duplicate_counts_per_value.index.tolist()
# top_100 = top_titles[:100]

# for title in top_100:
#     print(title)
    
# with open("top_100.pkl", "wb") as file:
#     pickle.dump(top_100, file)

In [6]:
# threshold = 5#num_users * 0.1
# pop_titles = list(duplicate_counts_per_value[duplicate_counts_per_value > threshold].index)
# my_titles = df.loc[df["User_id"] == 169695558, "Title"].tolist()
# # print(my_titles)

# print("pop titles len = ", len(pop_titles))
# print(pop_titles)
# print("my titles len = ", len(my_titles))
# titles = list(set(pop_titles))# + my_titles))

# # #remove Harry Potter titles:
# # titles = [s for s in titles if "Harry Potter" not in s]

# num_titles = len(titles)


# print(titles)
# # print(titles)
# print("num_titles =", num_titles)

# # ratings = np.full((num_users, num_titles), None)
# ratings = np.zeros((num_users, num_titles))

# for index, row in df.iterrows():
#     if row['Title'] in titles:
#         try:
#             ratings[user_ids.index(row['User_id']), titles.index(row["Title"])] = int(row["Rating"])
# #             print("found ", row["Title"])
#         except:
#             pass
        
# print("ratings size = ", ratings.shape)
# ratings = ratings[~np.all(ratings == 0, axis=1)]
# print("ratings size = ", ratings.shape)
# # Save the list to a file
# with open("titles.pkl", "wb") as file:
#     pickle.dump(titles, file)

In [7]:
# # print(df['Suggest'])

# suggest = list(df['Suggest'])
# print(suggest)

# with open("suggest.pkl", "wb") as file:
#     suggest.dump(suggest, file)

In [8]:
# ratings_df = pd.DataFrame(ratings)
# print(ratings_df.shape)
# #delete users that don't have any of these ratings
# # ratings_df = ratings_df.loc[~(ratings_df == 0).all(axis=1)]
# ratings_df = ratings_df.loc[(ratings_df != 0).sum(axis=1) >= 4] #need at least 4 entries to stay
# print(ratings_df.shape)

# # Calculate percentage of non-zero elements
# percentage_nonzero = (np.count_nonzero(ratings_df) / ratings_df.size) * 100
# print("percentage_nonzero =", round(percentage_nonzero, 1), '%')

# # Save the list to a file
# with open("ratings_df.pkl", "wb") as file:
#     pickle.dump(ratings_df, file)


In [9]:
with open("titles.pkl", "rb") as file:
    titles = pickle.load(file)
    
with open("top_100.pkl", "rb") as file:
    top_100 = pickle.load(file)
    
with open("suggest.pkl", "rb") as file:
    suggest = pickle.load(file)
    
num_titles = len(titles)

# Load the list from the file
with open("ratings_df.pkl", "rb") as file:
    ratings_df = pickle.load(file)
    ratings = ratings_df.to_numpy()


In [10]:
# Convert the filled ratings data into a numpy array
ratings_matrix = ratings_df.values

# Initialize KNN (using user-based KNN)
import math
knn = NearestNeighbors(n_neighbors=50, metric='cosine')  # Using cosine similarity  math.ceil(num_users/10)
knn.fit(ratings_matrix)

with open("knn_model.pkl", "wb") as file:
    pickle.dump(knn, file)

# Example: Predict rating for User 0 on Item 2
user_id = 0  # Index of user in the matrix
item_id = 2  # Index of item to predict rating for

# Get the nearest neighbors for user 0 (excluding the user itself)
distances, indices = knn.kneighbors([ratings_matrix[user_id]])

pred_ratings_list = np.array([])
rankings_list = np.array([])
for item_id in range(num_titles):
    # Get the ratings for the neighbors on item 2
    neighbor_ratings = np.array([ratings_matrix[i, item_id] for i in indices[0] if not np.isnan(ratings_matrix[i, item_id])])

    predicted_rating = np.mean(neighbor_ratings[np.nonzero(neighbor_ratings)])
    rankings = np.sum(neighbor_ratings[np.nonzero(neighbor_ratings)])

#     print(predicted_rating)

    pred_ratings_list = np.append(pred_ratings_list, predicted_rating)
    rankings_list = np.append(rankings_list, rankings)
#     print(f"Predicted rating for User {user_id} on Item {item_id}: {predicted_rating}")
    
best_book_rating = np.max(pred_ratings_list)
best_book_idx = np.argmax(pred_ratings_list)

sorted_indices = np.argsort(pred_ratings_list)[::-1]
print("Top books are:")
n = 1
for i, idx in enumerate(sorted_indices): 
    if (ratings[0, idx] > 0) or (np.isnan(pred_ratings_list[idx])):
        pass
    else:
        print("#", (n) , titles[idx], "Rating:", round(pred_ratings_list[idx], 1))
        n+=1

/root/miniconda3/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/root/miniconda3/lib/python3.9/site-packages/numpy/core/_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


Top books are:
# 1 The Good Earth
        (House of Earth, #1) Rating: 5.0
# 2 Siege and Storm
        (The Shadow and Bone Trilogy, #2) Rating: 5.0
# 3 Marley and Me: Life and Love With the Worldâ€™s Worst Dog Rating: 5.0
# 4 The Witch of Blackbird Pond Rating: 5.0
# 5 Homegoing Rating: 5.0
# 6 Corduroy Rating: 5.0
# 7 Cold Sassy Tree Rating: 5.0
# 8 Bride Rating: 5.0
# 9 The Passage
        (The Passage, #1) Rating: 5.0
# 10 The Thorn Birds Rating: 5.0
# 11 Franny and Zooey Rating: 5.0
# 12 Cannery Row
        (Cannery Row, #1) Rating: 5.0
# 13 Peter Pan Rating: 5.0
# 14 Small Great Things Rating: 5.0
# 15 Orphan Train Rating: 5.0
# 16 November 9 Rating: 5.0
# 17 The Princess Diaries
        (The Princess Diaries, #1) Rating: 5.0
# 18 One Day in the Life of Ivan Denisovich Rating: 5.0
# 19 The Testaments
        (The Handmaid's Tale, #2) Rating: 5.0
# 20 The Tale of Despereaux Rating: 5.0
# 21 The Girl Who Played with Fire
        (Millennium, #2) Rating: 5.0
# 22 Stone of Tears
    

# 425 The Forgotten Garden Rating: 4.0
# 426 The Importance of Being Earnest Rating: 4.0
# 427 The Tale of Peter Rabbit
        (World of Beatrix Potter, #1) Rating: 4.0
# 428 The Shining
        (The Shining, #1) Rating: 4.0
# 429 City of Fallen Angels
        (The Mortal Instruments, #4) Rating: 4.0
# 430 The Things They Carried Rating: 4.0
# 431 Shadow and Bone
        (The Shadow and Bone Trilogy, #1) Rating: 4.0
# 432 Three Cups of Tea: One Man's Mission to Promote Peace ... One School at a Time Rating: 4.0
# 433 Pandemonium
        (Delirium, #2) Rating: 4.0
# 434 The Mysterious Benedict Society
        (The Mysterious Benedict Society, #1) Rating: 4.0
# 435 Goodnight Moon Rating: 4.0
# 436 The Horse and His Boy
        (Chronicles of Narnia, #5) Rating: 4.0
# 437 Thirteen Reasons Why Rating: 4.0
# 438 The Silver Chair
        (Chronicles of Narnia, #4) Rating: 4.0
# 439 A Darker Shade of Magic
        (Shades of Magic, #1) Rating: 4.0
# 440 Les MisÃ©rables Rating: 4.0
# 441 The 

        (Earthseed, #1) Rating: 2.0
# 772 Left Behind
        (Left Behind, #1) Rating: 2.0
# 773 The Arabian Nights Rating: 2.0
# 774 Choke Rating: 2.0
# 775 Project Hail Mary Rating: 2.0
# 776 Beautiful Disaster
        (Beautiful, #1) Rating: 2.0
# 777 Lucky Rating: 2.0
# 778 Beautiful Creatures
        (Caster Chronicles, #1) Rating: 2.0
# 779 The Tipping Point: How Little Things Can Make a Big Difference Rating: 2.0
# 780 On the Road Rating: 2.0
# 781 The Once and Future King Rating: 1.0
# 782 The Audacity of Hope: Thoughts on Reclaiming the American Dream Rating: 1.0
# 783 I Know Why the Caged Bird Sings
        (Maya Angelou's Autobiography, #1) Rating: 1.0
# 784 As You Like It Rating: 1.0
# 785 Lies My Teacher Told Me: Everything Your American History Textbook Got Wrong Rating: 1.0
# 786 The Last Song Rating: 1.0
# 787 Catch-22 Rating: 1.0
# 788 Killing Lincoln: The Shocking Assassination that Changed America Forever Rating: 1.0
# 789 Cutting for Stone Rating: 1.0
# 790 Twiligh

In [11]:
#make knn for the similar user part
# Initialize KNN (using user-based KNN)
# import math
knn_30 = NearestNeighbors(n_neighbors=50, metric='cosine')  # Using cosine similarity
knn_30.fit(ratings_matrix)

with open("knn_model_30.pkl", "wb") as file:
    pickle.dump(knn_30, file)

In [12]:
rankings_list

best_book_rating = np.max(rankings_list)
best_book_idx = np.argmax(rankings_list)

sorted_indices = np.argsort(rankings_list)[::-1]
print("Top books are:")
for i, idx in enumerate(sorted_indices): 
    print("#", (i+1) , titles[idx], "Rating:", round(pred_ratings_list[idx], 1), ". Ranking:", rankings_list[idx])

Top books are:
# 1 Harry Potter and the Prisoner of Azkaban
        (Harry Potter, #3) Rating: 4.7 . Ranking: 212.0
# 2 Harry Potter and the Deathly Hallows
        (Harry Potter, #7) Rating: 4.8 . Ranking: 207.0
# 3 Harry Potter and the Goblet of Fire
        (Harry Potter, #4) Rating: 4.8 . Ranking: 205.0
# 4 Harry Potter and the Half-Blood Prince
        (Harry Potter, #6) Rating: 4.6 . Ranking: 203.0
# 5 Harry Potter and the Order of the Phoenix
        (Harry Potter, #5) Rating: 4.6 . Ranking: 202.0
# 6 Harry Potter and the Chamber of Secrets
        (Harry Potter, #2) Rating: 4.6 . Ranking: 199.0
# 7 Harry Potter and the Sorcerer's Stone
        (Harry Potter, #1) Rating: 4.6 . Ranking: 193.0
# 8 The Hunger Games
        (The Hunger Games, #1) Rating: 4.8 . Ranking: 176.0
# 9 Catching Fire
        (The Hunger Games, #2) Rating: 4.6 . Ranking: 147.0
# 10 Mockingjay
        (The Hunger Games, #3) Rating: 4.5 . Ranking: 134.0
# 11 Twilight
        (The Twilight Saga, #1) Rating: 3.9

# 220 Mere Christianity Rating: 5.0 . Ranking: 10.0
# 221 Living Dead in Dallas
        (Sookie Stackhouse, #2) Rating: 5.0 . Ranking: 10.0
# 222 Forever in Blue: The Fourth Summer of the Sisterhood
        (Sisterhood, #4) Rating: 3.3 . Ranking: 10.0
# 223 The Complete Persepolis Rating: 5.0 . Ranking: 10.0
# 224 Beloved
        (Beloved Trilogy, #1) Rating: 3.3 . Ranking: 10.0
# 225 Harry Potter Series Box Set
        (Harry Potter, #1-7) Rating: 5.0 . Ranking: 10.0
# 226 Dead Until Dark
        (Sookie Stackhouse, #1) Rating: 5.0 . Ranking: 10.0
# 227 Ten Big Ones
        (Stephanie Plum, #10) Rating: 3.3 . Ranking: 10.0
# 228 Anna Karenina Rating: 3.3 . Ranking: 10.0
# 229 The Angel Experiment
        (Maximum Ride, #1) Rating: 5.0 . Ranking: 10.0
# 230 From Dead to Worse
        (Sookie Stackhouse, #8) Rating: 5.0 . Ranking: 10.0
# 231 Stone of Tears
        (Sword of Truth, #2) Rating: 5.0 . Ranking: 10.0
# 232 The Passage
        (The Passage, #1) Rating: 5.0 . Ranking: 10.0
# 2

        (Letters of Enchantment, #1) Rating: 5.0 . Ranking: 5.0
# 472 Moon Called
        (Mercy Thompson, #1) Rating: 5.0 . Ranking: 5.0
# 473 Fire
        (Graceling Realm, #2) Rating: 5.0 . Ranking: 5.0
# 474 Dr. Seuss's Green Eggs and Ham: For Soprano, Boy Soprano, and Orchestra Rating: 5.0 . Ranking: 5.0
# 475 The Opal Deception
        (Artemis Fowl, #4) Rating: 5.0 . Ranking: 5.0
# 476 Maus I: A Survivor's Tale: My Father Bleeds History
        (Maus, #1) Rating: 5.0 . Ranking: 5.0
# 477 Dress Your Family in Corduroy and Denim Rating: 5.0 . Ranking: 5.0
# 478 Funny Story Rating: 5.0 . Ranking: 5.0
# 479 Demon Copperhead Rating: 5.0 . Ranking: 5.0
# 480 A Kingdom of Flesh and Fire
        (Blood and Ash, #2) Rating: 5.0 . Ranking: 5.0
# 481 Howlâ€™s Moving Castle
        (Howlâ€™s Moving Castle, #1) Rating: 5.0 . Ranking: 5.0
# 482 The Girl You Left Behind Rating: 5.0 . Ranking: 5.0
# 483 So Long, and Thanks for All the Fish
        (Hitchhiker's Guide to the Galaxy, #4) Rating: 

# 746 The Last to Vanish Rating: 3.0 . Ranking: 3.0
# 747 Daisy Darker Rating: 3.0 . Ranking: 3.0
# 748 Blink: The Power of Thinking Without Thinking Rating: 3.0 . Ranking: 3.0
# 749 Where the Heart Is Rating: 3.0 . Ranking: 3.0
# 750 The Guest List Rating: 3.0 . Ranking: 3.0
# 751 A Man Called Ove Rating: 3.0 . Ranking: 3.0
# 752 The Selection
        (The Selection, #1) Rating: 3.0 . Ranking: 3.0
# 753 The Thirteenth Tale Rating: 3.0 . Ranking: 3.0
# 754 Storm Front
        (The Dresden Files, #1) Rating: 3.0 . Ranking: 3.0
# 755 Normal People Rating: 3.0 . Ranking: 3.0
# 756 Caught Rating: 3.0 . Ranking: 3.0
# 757 Educated Rating: 3.0 . Ranking: 3.0
# 758 Make Way for Ducklings Rating: 3.0 . Ranking: 3.0
# 759 The Elite
        (The Selection, #2) Rating: 3.0 . Ranking: 3.0
# 760 Snow Falling on Cedars Rating: 3.0 . Ranking: 3.0
# 761 In Her Shoes Rating: 3.0 . Ranking: 3.0
# 762 The Remains of the Day Rating: 3.0 . Ranking: 3.0
# 763 The Gift of the Magi Rating: 3.0 . Ranking: 3.0


# 1004 Revolutionary Road Rating: nan . Ranking: 0.0
# 1005 The Amityville Horror Rating: nan . Ranking: 0.0
# 1006 If We Were Villains Rating: nan . Ranking: 0.0
# 1007 Reading Lolita in Tehran: A Memoir in Books Rating: nan . Ranking: 0.0
# 1008 The Mystery Guest
        (Molly the Maid, #2) Rating: nan . Ranking: 0.0
# 1009 Invisible Man Rating: nan . Ranking: 0.0
# 1010 Cryptonomicon Rating: nan . Ranking: 0.0
# 1011 The Devil in the White City: Murder, Magic, and Madness at the Fair That Changed America Rating: nan . Ranking: 0.0
# 1012 The Very Secret Society of Irregular Witches Rating: nan . Ranking: 0.0
# 1013 The Idiot Rating: nan . Ranking: 0.0
# 1014 Edge of Eternity
        (The Century Trilogy, #3) Rating: nan . Ranking: 0.0
# 1015 Glass Sword
        (Red Queen, #2) Rating: nan . Ranking: 0.0
# 1016 Outliers: The Story of Success Rating: nan . Ranking: 0.0
# 1017 Demian Rating: nan . Ranking: 0.0
# 1018 A Connecticut Yankee in King Arthur's Court Rating: nan . Ranking: 0

        (Thursday Murder Club, #4) Rating: nan . Ranking: 0.0
# 1263 The Woman in Cabin 10 Rating: nan . Ranking: 0.0
# 1264 The Other Mothers Rating: nan . Ranking: 0.0
# 1265 The Hitchhiker’s Guide to the Galaxy
        (Hitchhiker's Guide to the Galaxy, #1) Rating: nan . Ranking: 0.0
# 1266 The Rules of Magic
        (Practical Magic, #0.2) Rating: nan . Ranking: 0.0
# 1267 Illusions: The Adventures of a Reluctant Messiah Rating: nan . Ranking: 0.0
# 1268 Skin Game
        (The Dresden Files, #15) Rating: nan . Ranking: 0.0
# 1269 Reamde Rating: nan . Ranking: 0.0
# 1270 The Long Dark Tea-Time of the Soul
        (Dirk Gently, #2) Rating: nan . Ranking: 0.0
# 1271 Rock Paper Scissors Rating: nan . Ranking: 0.0
# 1272 The Night Watchman Rating: nan . Ranking: 0.0
# 1273 Equal Rites
        (Discworld, #3; Witches, #1) Rating: nan . Ranking: 0.0
# 1274 The Witching Hour
        (Lives of the Mayfair Witches, #1) Rating: nan . Ranking: 0.0
# 1275 The Secret Book of Flora Lea Rating: na

# 1501 The Heart Is a Lonely Hunter Rating: nan . Ranking: 0.0
# 1502 The Hobbit Rating: nan . Ranking: 0.0
# 1503 Recursion Rating: nan . Ranking: 0.0
# 1504 Rose Madder Rating: nan . Ranking: 0.0
# 1505 The Searcher Rating: nan . Ranking: 0.0
# 1506 The 7 Habits of Highly Effective People: Powerful Lessons in Personal Change Rating: nan . Ranking: 0.0
# 1507 A Great and Terrible Beauty
        (Gemma Doyle, #1) Rating: nan . Ranking: 0.0
# 1508 The Peach Keeper Rating: nan . Ranking: 0.0
# 1509 The Invisible Life of Addie LaRue Rating: nan . Ranking: 0.0
# 1510 The Hours Rating: nan . Ranking: 0.0
# 1511 Dragonflight
        (Dragonriders of Pern, #1) Rating: nan . Ranking: 0.0
# 1512 The Book of Cold Cases Rating: nan . Ranking: 0.0
# 1513 The God Delusion Rating: nan . Ranking: 0.0
# 1514 The Glass Hotel Rating: nan . Ranking: 0.0
# 1515 The Woman in Black Rating: nan . Ranking: 0.0
# 1516 The Unmaking of June Farrow Rating: nan . Ranking: 0.0
# 1517 The Last One at the Wedding Rat

# 1738 Thank You for Listening Rating: nan . Ranking: 0.0
# 1739 Trust Rating: nan . Ranking: 0.0
# 1740 Middlemarch Rating: nan . Ranking: 0.0
# 1741 Are You There God? Itâ€™s Me, Margaret Rating: nan . Ranking: 0.0
# 1742 It Happened One Summer
        (Bellinger Sisters, #1) Rating: nan . Ranking: 0.0
# 1743 The House in the Cerulean Sea
        (Cerulean Chronicles, #1) Rating: nan . Ranking: 0.0
# 1744 The Other Black Girl Rating: nan . Ranking: 0.0
# 1745 The Reversal
        (The Lincoln Lawyer, #3; Harry Bosch Universe, #22) Rating: nan . Ranking: 0.0
# 1746 The Black Swan: The Impact of the Highly Improbable Rating: nan . Ranking: 0.0
# 1747 Tell the Wolves I'm Home Rating: nan . Ranking: 0.0
# 1748 Four Past Midnight Rating: nan . Ranking: 0.0
# 1749 The Windup Girl Rating: nan . Ranking: 0.0
# 1750 Go, Dog. Go! Rating: nan . Ranking: 0.0
# 1751 Extremely Loud & Incredibly Close Rating: nan . Ranking: 0.0
# 1752 The Three-Body Problem
        (Remembrance of Earthâ€™s Past, #

# 2004 The Wife Between Us Rating: nan . Ranking: 0.0
# 2005 Spirit Bound
        (Vampire Academy, #5) Rating: nan . Ranking: 0.0
# 2006 Death on the Nile
        (Hercule Poirot, #18) Rating: nan . Ranking: 0.0
# 2007 Pieces of Her
        (Andrea Oliver, #1) Rating: nan . Ranking: 0.0
# 2008 The Black Echo
        (Harry Bosch, #1; Harry Bosch Universe, #1) Rating: nan . Ranking: 0.0
# 2009 Meet Me at the Lake Rating: nan . Ranking: 0.0
# 2010 Dreams from My Father: A Story of Race and Inheritance Rating: nan . Ranking: 0.0
# 2011 Wrong Place Wrong Time Rating: nan . Ranking: 0.0
# 2012 Inferno
        (Robert Langdon, #4) Rating: nan . Ranking: 0.0
# 2013 You
        (You, #1) Rating: nan . Ranking: 0.0
# 2014 The Light Fantastic
        (Discworld, #2; Rincewind, #2) Rating: nan . Ranking: 0.0
# 2015 A Fine Balance Rating: nan . Ranking: 0.0
# 2016 The Sugar Queen Rating: nan . Ranking: 0.0
# 2017 The One and Only Ivan
        (The One and Only #1) Rating: nan . Ranking: 0.0
# 201

# 2266 If You Give a Mouse a Cookie Rating: nan . Ranking: 0.0
# 2267 Don Quixote Rating: nan . Ranking: 0.0
# 2268 Memory Man
        (Amos Decker, #1) Rating: nan . Ranking: 0.0
# 2269 Desperation Rating: nan . Ranking: 0.0
# 2270 Little Bee Rating: nan . Ranking: 0.0
# 2271 Hidden Valley Road: Inside the Mind of an American Family Rating: nan . Ranking: 0.0
# 2272 Preludes & Nocturnes
        (The Sandman, #1) Rating: nan . Ranking: 0.0
# 2273 In the Time of the Butterflies Rating: nan . Ranking: 0.0
# 2274 The Interestings Rating: nan . Ranking: 0.0
# 2275 The Last of the Mohicans
        (The Leatherstocking Tales, #2) Rating: nan . Ranking: 0.0
# 2276 The Glass Menagerie Rating: nan . Ranking: 0.0
# 2277 The Likeness
        (Dublin Murder Squad, #2) Rating: nan . Ranking: 0.0
# 2278 Into Thin Air: A Personal Account of the Mt. Everest Disaster Rating: nan . Ranking: 0.0
# 2279 Finding Cinderella
        (Hopeless, #2.5) Rating: nan . Ranking: 0.0
# 2280 Death Masks
        (The 

In [13]:
#give a list sorted out with books you've already read:
# sorted_indices = np.argsort(pred_ratings_list)[::-1]
print("Top books are:")
list_num = 1
for idx in sorted_indices: 
#     print("ratings_matrix[user_id, idx]= ", ratings_matrix[user_id, idx])
    if (ratings_matrix[user_id, idx] > 0) or (np.isnan(pred_ratings_list[idx])):
        continue
    print("#", list_num , titles[idx], "Rating:", round(pred_ratings_list[idx], 1))
    list_num += 1

Top books are:
# 1 Harry Potter and the Sorcerer's Stone
        (Harry Potter, #1) Rating: 4.6
# 2 The da Vinci Code
        (Robert Langdon, #2) Rating: 4.4
# 3 Eclipse
        (The Twilight Saga, #3) Rating: 4.1
# 4 Breaking Dawn
        (The Twilight Saga, #4) Rating: 3.9
# 5 The Great Gatsby Rating: 3.7
# 6 Pride and Prejudice Rating: 4.2
# 7 The Kite Runner Rating: 4.7
# 8 The Lion, the Witch and the Wardrobe
        (Chronicles of Narnia, #1) Rating: 4.2
# 9 The Host
        (The Host, #1) Rating: 4.7
# 10 The Sea of Monsters
        (Percy Jackson and the Olympians, #2) Rating: 4.0
# 11 The Lovely Bones Rating: 3.5
# 12 The Book Thief Rating: 4.8
# 13 The Help Rating: 4.8
# 14 Lord of the Flies Rating: 3.1
# 15 The Diary of a Young Girl Rating: 4.6
# 16 The Hobbit, or There and Back Again
        (The Lord of the Rings, #0) Rating: 4.6
# 17 The Count of Monte Cristo Rating: 4.4
# 18 The Last Olympian
        (Percy Jackson and the Olympians, #5) Rating: 4.4
# 19 Number the Star

# 317 North and South Rating: 5.0
# 318 Rosencrantz and Guildenstern Are Dead Rating: 5.0
# 319 Cannery Row
        (Cannery Row, #1) Rating: 5.0
# 320 The Witch of Blackbird Pond Rating: 5.0
# 321 Somebody's Daughter Rating: 5.0
# 322 The Wise Manâ€™s Fear
        (The Kingkiller Chronicle, #2) Rating: 5.0
# 323 The Monster at the End of this Book Rating: 5.0
# 324 Finders Keepers
        (Bill Hodges Trilogy, #2) Rating: 5.0
# 325 The Last Lecture Rating: 5.0
# 326 The Princess Diaries
        (The Princess Diaries, #1) Rating: 5.0
# 327 Nine Stories Rating: 5.0
# 328 The Summer I Turned Pretty
        (Summer, #1) Rating: 5.0
# 329 Cold Sassy Tree Rating: 5.0
# 330 Alice's Adventures in Wonderland Rating: 5.0
# 331 The Cruel Prince
        (The Folk of the Air, #1) Rating: 5.0
# 332 Love, Theoretically Rating: 5.0
# 333 The Little Prince Rating: 5.0
# 334 The Invention of Wings Rating: 5.0
# 335 East of Eden Rating: 5.0
# 336 DÃ©jÃ  Dead
        (Temperance Brennan, #1) Rating: 5.0


# 674 Perfume: The Story of a Murderer Rating: 3.0
# 675 Under the Banner of Heaven: A Story of Violent Faith Rating: 3.0
# 676 Darkly Dreaming Dexter
        (Dexter, #1) Rating: 3.0
# 677 The Forest of Hands and Teeth
        (The Forest of Hands and Teeth, #1) Rating: 3.0
# 678 Daisy Darker Rating: 3.0
# 679 Blink: The Power of Thinking Without Thinking Rating: 3.0
# 680 Where the Heart Is Rating: 3.0
# 681 The Guest List Rating: 3.0
# 682 A Man Called Ove Rating: 3.0
# 683 The Selection
        (The Selection, #1) Rating: 3.0
# 684 The Thirteenth Tale Rating: 3.0
# 685 Storm Front
        (The Dresden Files, #1) Rating: 3.0
# 686 Normal People Rating: 3.0
# 687 Caught Rating: 3.0
# 688 Educated Rating: 3.0
# 689 Make Way for Ducklings Rating: 3.0
# 690 The Elite
        (The Selection, #2) Rating: 3.0
# 691 Snow Falling on Cedars Rating: 3.0
# 692 In Her Shoes Rating: 3.0
# 693 The Remains of the Day Rating: 3.0
# 694 The Gift of the Magi Rating: 3.0
# 695 The Restaurant at the End

In [14]:
idx = 8
this_ratings = ratings_matrix[indices[0,idx]]
print(this_ratings)

my_ratings = ratings_matrix[indices[0,0]]
print(my_ratings)

for i, rating in enumerate(this_ratings):
    if rating > 0 and my_ratings[i]>0:
        print("-", titles[i], ", their Rating:", rating, " My Rating:", my_ratings[i])

# print(distances, indices[-1,-1])

[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
- Harry Potter and the Chamber of Secrets
        (Harry Potter, #2) , their Rating: 5.0  My Rating: 5.0
- The Outsiders , their Rating: 5.0  My Rating: 4.0
- To Kill a Mockingbird , their Rating: 3.0  My Rating: 5.0
- Divergent
        (Divergent, #1) , their Rating: 4.0  My Rating: 5.0
- The Lightning Thief
        (Percy Jackson and the Olympians, #1) , their Rating: 4.0  My Rating: 5.0
- Harry Potter and the Prisoner of Azkaban
        (Harry Potter, #3) , their Rating: 5.0  My Rating: 5.0
- Harry Potter and the Goblet of Fire
        (Harry Potter, #4) , their Rating: 5.0  My Rating: 5.0
- Harry Potter and the Order of the Phoenix
        (Harry Potter, #5) , their Rating: 5.0  My Rating: 5.0
- Harry Potter and the Deathly Hallows
        (Harry Potter, #7) , their Rating: 5.0  My Rating: 5.0
- The Hunger Games
        (The Hunger Games, #1) , their Rating: 4.0  My Rating: 5.0
- Harry Potter and the Philosopher’s Stone
        (Harry

In [15]:
#find most similar books using cosine similarity
import numpy as np
import pandas as pd
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics.pairwise import cosine_similarity

similarity_matrix = cosine_similarity(ratings_matrix.T)

similarity_df = pd.DataFrame(similarity_matrix, index=titles, columns=titles)

# Function to get k nearest neighbors for a movie
def get_similar_book(book_name, k=3):
    similar_book = similarity_df[book_name].sort_values(ascending=False)[1:k+1]
    return similar_book

book_name = 'First Lie Wins'
print("\nTop 5 similar book to", book_name, ":")
print(get_similar_book(book_name, k=5))


Top 5 similar book to First Lie Wins :
She's Not Sorry                0.602133
Listen for the Lie             0.591416
Darling Girls                  0.578486
The Last One at the Wedding    0.568676
The Fury                       0.563608
Name: First Lie Wins, dtype: float64


In [16]:
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.impute import SimpleImputer

# Example user rating data (rows = users, columns = items)
ratings_df = pd.DataFrame(ratings)

# Step 1: Handle missing values using imputation (we will fill missing values with the mean rating)
imputer = SimpleImputer(strategy='mean')
ratings_filled = imputer.fit_transform(ratings_df)

# Step 2: Apply KMeans clustering
# We will use 2 clusters as an example
kmeans = KMeans(n_clusters=int(num_users/10), random_state=42)
clusters = kmeans.fit_predict(ratings_filled)

# Step 3: Add the cluster labels to the original DataFrame
ratings_df['Cluster'] = clusters

# # Print the user ratings with cluster assignments
# print("\nUser Ratings with Clusters:")
# print(ratings_df)

# # Step 4: Print the cluster centers (the centroid of each cluster)
# print("\nCluster Centers (Centroids):")
# print(kmeans.cluster_centers_)


user_id = 0
# print(clusters)

cluster_this_user = clusters[user_id]
# print(cluster_this_user)

pred_ratings_list = kmeans.cluster_centers_[user_id]
# print(pred_ratings_list)
# for i in range(len())

#give a list sorted out with books you've already read:
sorted_indices = np.argsort(pred_ratings_list)[::-1]
print("Top books are:")
list_num = 1
for idx in sorted_indices: 
#     print("ratings_matrix[user_id, idx]= ", ratings_matrix[user_id, idx])
    if (ratings_matrix[user_id, idx] > 0) or (np.isnan(pred_ratings_list[idx])) :
        continue
    print("#", list_num , titles[idx], "Rating:", round(pred_ratings_list[idx], 1))
    list_num += 1

/root/miniconda3/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Top books are:
# 1 Lord of Chaos
        (The Wheel of Time, #6) Rating: 4.8
# 2 The Eye of the World
        (The Wheel of Time, #1) Rating: 4.8
# 3 The Path of Daggers
        (The Wheel of Time, #8) Rating: 4.6
# 4 A Crown of Swords
        (The Wheel of Time, #7) Rating: 4.4
# 5 The Fires of Heaven
        (The Wheel of Time, #5) Rating: 3.8
# 6 The Great Hunt
        (The Wheel of Time, #2) Rating: 3.8
# 7 The Dragon Reborn
        (The Wheel of Time, #3) Rating: 3.8
# 8 Knife of Dreams
        (The Wheel of Time, #11) Rating: 3.6
# 9 Crossroads of Twilight
        (The Wheel of Time, #10) Rating: 3.6
# 10 The Shadow Rising
        (The Wheel of Time, #4) Rating: 3.6
# 11 The Gathering Storm
        (The Wheel of Time, #12) Rating: 3.6
# 12 Winter's Heart
        (The Wheel of Time, #9) Rating: 3.4
# 13 Good Omens: The Nice and Accurate Prophecies of Agnes Nutter, Witch Rating: 2.0
# 14 Hot Six
        (Stephanie Plum, #6) Rating: 1.8
# 15 Towers of Midnight
        (The Wheel of 

# 257 Dead Ever After
        (Sookie Stackhouse, #13) Rating: 0.0
# 258 What the Wind Knows Rating: 0.0
# 259 The Summoning
        (Darkest Powers, #1) Rating: 0.0
# 260 Stories I Only Tell My Friends Rating: 0.0
# 261 Speaker for the Dead
        (Ender's Saga, #2) Rating: 0.0
# 262 A Touch of Dead Rating: 0.0
# 263 Simple Genius
        (Sean King & Michelle Maxwell, #3) Rating: 0.0
# 264 Curious George Rating: 0.0
# 265 Bye, Baby Rating: 0.0
# 266 That Summer Rating: 0.0
# 267 Where the Crawdads Sing Rating: 0.0
# 268 A Confederacy of Dunces Rating: 0.0
# 269 Fables, Vol. 1: Legends in Exile Rating: 0.0
# 270 Everyone in My Family Has Killed Someone
        (Ernest Cunningham, #1) Rating: 0.0
# 271 The Night Shift Rating: 0.0
# 272 The Paris Wife Rating: 0.0
# 273 Finger Lickin' Fifteen
        (Stephanie Plum, #15) Rating: 0.0
# 274 The Dead Romantics Rating: 0.0
# 275 The Velveteen Rabbit Rating: 0.0
# 276 The Mark of Athena
        (The Heroes of Olympus, #3) Rating: 0.0
# 277 

# 690 Not in Love Rating: 0.0
# 691 The Rosie Project
        (Don Tillman, #1) Rating: 0.0
# 692 Number the Stars Rating: 0.0
# 693 Outline Rating: 0.0
# 694 The It Girl Rating: 0.0
# 695 Writers & Lovers Rating: 0.0
# 696 The Only One Left Rating: 0.0
# 697 Bring Me Back Rating: 0.0
# 698 Small Favor
        (The Dresden Files, #10) Rating: 0.0
# 699 A Rule Against Murder
        (Chief Inspector Armand Gamache, #4) Rating: 0.0
# 700 A Time for Mercy
        (Jake Brigance, #3) Rating: 0.0
# 701 Romantic Comedy Rating: 0.0
# 702 Brida Rating: 0.0
# 703 The Lost Apothecary Rating: 0.0
# 704 The Happy Ever After Playlist
        (The Friend Zone, #2) Rating: 0.0
# 705 The Lacuna Rating: 0.0
# 706 The Trespasser
        (Dublin Murder Squad, #6) Rating: 0.0
# 707 Lady Chatterley's Lover Rating: 0.0
# 708 Go Set a Watchman Rating: 0.0
# 709 The Valley of Horses
        (Earth's Children, #2) Rating: 0.0
# 710 All the Pretty Horses
        (The Border Trilogy, #1) Rating: 0.0
# 711 Soulle

# 990 The Girl You Left Behind Rating: 0.0
# 991 Tara Road Rating: 0.0
# 992 A Court of Frost and Starlight
        (A Court of Thorns and Roses, #3.5) Rating: 0.0
# 993 Mort
        (Discworld, #4; Death, #1) Rating: 0.0
# 994 The Lies of Locke Lamora
        (Gentleman Bastard, #1) Rating: 0.0
# 995 The Soulmate Rating: 0.0
# 996 Red at the Bone Rating: 0.0
# 997 Little Earthquakes Rating: 0.0
# 998 Red Seas Under Red Skies
        (Gentleman Bastard, #2) Rating: 0.0
# 999 The 7 1/2 Deaths of Evelyn Hardcastle Rating: 0.0
# 1000 The Casual Vacancy Rating: 0.0
# 1001 Project Hail Mary Rating: 0.0
# 1002 The True Love Experiment Rating: 0.0
# 1003 A Tree Grows in Brooklyn Rating: 0.0
# 1004 A Hunger Like No Other
        (Immortals After Dark, #1) Rating: 0.0
# 1005 The Friday Night Knitting Club
        (Friday Night Knitting Club, #1) Rating: 0.0
# 1006 The Name of the Rose Rating: 0.0
# 1007 The Time Traveler's Wife Rating: 0.0
# 1008 Small Things Like These Rating: 0.0
# 1009 The M

# 1304 True Believer
        (Jeremy Marsh & Lexie Darnell, #1) Rating: 0.0
# 1305 Sunshine Rating: 0.0
# 1306 Beautiful Ruins Rating: 0.0
# 1307 Norse Mythology Rating: 0.0
# 1308 Handle with Care Rating: 0.0
# 1309 Harry Potter and the Cursed Child: Parts One and Two
        (Harry Potter, #8) Rating: 0.0
# 1310 Night Road Rating: 0.0
# 1311 The Many Lives of Mama Love: A Memoir of Lying, Stealing, Writing, and Healing Rating: 0.0
# 1312 Once Upon a Broken Heart
        (Once Upon a Broken Heart, #1) Rating: 0.0
# 1313 The God of the Woods Rating: 0.0
# 1314 The Cat in the Hat
        (Cat in the Hat, #1) Rating: 0.0
# 1315 Go, Dog. Go! Rating: 0.0
# 1316 I Know This Much Is True Rating: 0.0
# 1317 Lover Mine
        (Black Dagger Brotherhood, #8) Rating: 0.0
# 1318 2001: A Space Odyssey
        (Space Odyssey, #1) Rating: 0.0
# 1319 My Name Is Lucy Barton
        (Amgash, #1) Rating: 0.0
# 1320 The Son Rating: 0.0
# 1321 The Professor and the Madman: A Tale of Murder, Insanity and t

        (Earthsea Cycle, #3) Rating: 0.0
# 1591 The Clan of the Cave Bear
        (Earth's Children, #1) Rating: 0.0
# 1592 Confess Rating: 0.0
# 1593 The Adventures of Sherlock Holmes
        (Sherlock Holmes, #3) Rating: 0.0
# 1594 The Golden Couple Rating: 0.0
# 1595 The Last Letter from Your Lover Rating: 0.0
# 1596 Another Brooklyn Rating: 0.0
# 1597 West With Giraffes Rating: 0.0
# 1598 The Buried Giant Rating: 0.0
# 1599 Howlâ€™s Moving Castle
        (Howlâ€™s Moving Castle, #1) Rating: 0.0
# 1600 A Talent for Murder
        (Henry Kimball/Lily Kintner, #3) Rating: 0.0
# 1601 The Girls' Guide to Hunting and Fishing Rating: 0.0
# 1602 Nine Perfect Strangers Rating: 0.0
# 1603 Wizard and Glass
        (The Dark Tower, #4) Rating: 0.0
# 1604 The Life-Changing Magic of Tidying Up: The Japanese Art of Decluttering and Organizing Rating: 0.0
# 1605 The Weight of Silence Rating: 0.0
# 1606 Dry Rating: 0.0
# 1607 Starling House Rating: 0.0
# 1608 28 Summers Rating: 0.0
# 1609 The Thirt

# 1893 The Witching Hour
        (Lives of the Mayfair Witches, #1) Rating: -0.0
# 1894 Before They Are Hanged
        (The First Law, #2) Rating: -0.0
# 1895 An Absolutely Remarkable Thing
        (The Carls, #1) Rating: -0.0
# 1896 Annihilation
        (Southern Reach, #1) Rating: -0.0
# 1897 Echo Burning
        (Jack Reacher, #5) Rating: -0.0
# 1898 The Last Word Rating: -0.0
# 1899 Pieces of Her
        (Andrea Oliver, #1) Rating: -0.0
# 1900 We Were Never Here Rating: -0.0
# 1901 The Year of Living Biblically: One Man's Humble Quest to Follow the Bible as Literally as Possible Rating: -0.0
# 1902 The Diamond Age: Or, a Young Lady's Illustrated Primer Rating: -0.0
# 1903 Notes from a Small Island Rating: -0.0
# 1904 Beastly
        (Beastly, #1) Rating: -0.0
# 1905 The Windup Girl Rating: -0.0
# 1906 On a Pale Horse
        (Incarnations of Immortality #1) Rating: -0.0
# 1907 Walking Disaster
        (Beautiful, #2) Rating: -0.0
# 1908 Paradise Lost Rating: -0.0
# 1909 Check & Mat

# 2199 House of Glass Rating: -0.0
# 2200 Under the Whispering Door Rating: -0.0
# 2201 This Is How It Always Is Rating: -0.0
# 2202 Wishful Drinking Rating: -0.0
# 2203 Origin
        (Robert Langdon, #5) Rating: -0.0
# 2204 Gone
        (Gone, #1) Rating: -0.0
# 2205 In the Unlikely Event Rating: -0.0
# 2206 The Rainbow Fish Rating: -0.0
# 2207 The Five Love Languages: How to Express Heartfelt Commitment to Your Mate Rating: -0.0
# 2208 Someone Else's Shoes Rating: -0.0
# 2209 Nothing to See Here Rating: -0.0
# 2210 The Great Divorce Rating: -0.0
# 2211 Confessions on the 7:45 Rating: -0.0
# 2212 The Body Keeps the Score: Brain, Mind, and Body in the Healing of Trauma Rating: -0.0
# 2213 The Final Gambit
        (The Inheritance Games, #3) Rating: -0.0
# 2214 The Housemaid Is Watching
        (The Housemaid, #3) Rating: -0.0
# 2215 And the Mountains Echoed Rating: -0.0
# 2216 Beloved
        (Beloved Trilogy, #1) Rating: -0.0
# 2217 The Plot
        (The Book Series, #1) Rating: -0.0

In [17]:
#doing masked autoencoder
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import KFold
from torch.utils.data import DataLoader, TensorDataset, random_split

# Mask for observed values (1 for observed, 0 for missing)
ratings_torch = torch.tensor(ratings).float()
mask = (ratings_torch != 0).float()
mask_tensor = torch.tensor(mask)


#Define autoencoder
class SparseAutoencoder(nn.Module):
    def __init__(self, num_items, latent_dim):
        super(SparseAutoencoder, self).__init__()
        self.encoder = nn.Linear(num_items, latent_dim)
        self.decoder = nn.Linear(latent_dim, num_items)
        
    def forward(self, x):
        encoded = torch.relu(self.encoder(x))
        decoded = self.decoder(encoded)
        # Scale sigmoid output to [1, 5]
        return 1 + 4 * torch.sigmoid(decoded)
        return decoded

    
#initialize the model
num_users, num_items = ratings_torch.shape
np.save("num_users.npy", np.array(num_users))
np.save("num_items.npy", np.array(num_items))
latent_dim = 100#int(num_items/4) # Number of latent features

model = SparseAutoencoder(num_items, latent_dim)
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Use MSE loss but only consider observed values
def masked_mse_loss(reconstructed, original, mask):
    loss = ((reconstructed - original) ** 2) * mask
    return loss.sum() / mask.sum()

#break up data into train and val
dataset = TensorDataset(ratings_torch, mask_tensor) #keeping the mask
print("ratings_torch shape =", ratings_torch.shape)
print(len(dataset))
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

print("len(train_loader) = ", len(train_loader))
print("len(val_loader) = ", len(val_loader))

#train the model
epochs = 5000
best_loss = 10e10
counter = 0
for epoch in range(epochs):
    train_loss = 0.0
    for inputs, mask in train_loader:
        model.train()
        optimizer.zero_grad()

        # Forward pass
        reconstructed = model(inputs)
        loss = masked_mse_loss(reconstructed, inputs, mask)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()

    train_loss /= len(train_loader)
    
    if (epoch + 1) % 10 == 0:
#         print(f"Epoch {epoch + 1}/{epochs}, Loss: {train_loss}")
        #check validation
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for inputs, mask in val_loader:
                outputs = model(inputs)
                loss = masked_mse_loss(outputs, inputs, mask)
                val_loss += loss.item()
                

        val_loss /= len(val_loader)
        

        print(f"Epoch {epoch + 1} - Train Loss: {train_loss:.4f} - Val Loss: {val_loss:.4f}")
        if val_loss < best_loss: #if improve then save
            #save model 
            torch.save(model.state_dict(), "model{}.pkl".format(latent_dim))
            best_loss = val_loss
            print("Model saved to model{}.pkl.".format(latent_dim))
            counter = 0
        else:
            counter += 10
            
    if counter > 200:
        print("Done training because of no improvement.")
        break
        
            


/tmp/ipykernel_1586/2558548069.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  mask_tensor = torch.tensor(mask)


ratings_torch shape = torch.Size([1266, 2482])
1266
len(train_loader) =  32
len(val_loader) =  8
Epoch 10 - Train Loss: 0.5942 - Val Loss: 1.1562
Model saved to model100.pkl.
Epoch 20 - Train Loss: 0.5495 - Val Loss: 1.1010
Model saved to model100.pkl.
Epoch 30 - Train Loss: 0.5518 - Val Loss: 1.1020
Epoch 40 - Train Loss: 0.5518 - Val Loss: 1.0711
Model saved to model100.pkl.
Epoch 50 - Train Loss: 0.5645 - Val Loss: 1.0658
Model saved to model100.pkl.
Epoch 60 - Train Loss: 0.5604 - Val Loss: 1.0698
Epoch 70 - Train Loss: 0.5622 - Val Loss: 1.0553
Model saved to model100.pkl.
Epoch 80 - Train Loss: 0.5577 - Val Loss: 1.0622
Epoch 90 - Train Loss: 0.5871 - Val Loss: 1.0622
Epoch 100 - Train Loss: 0.5995 - Val Loss: 1.0541
Model saved to model100.pkl.
Epoch 110 - Train Loss: 0.6015 - Val Loss: 1.0460
Model saved to model100.pkl.
Epoch 120 - Train Loss: 0.6008 - Val Loss: 1.0456
Model saved to model100.pkl.
Epoch 130 - Train Loss: 0.6015 - Val Loss: 1.0328
Model saved to model100.pkl.
E

In [18]:
num_users, num_items

(1266, 2482)

In [19]:
print(sum(sum(mask)))
print(mask.shape)
print(mask.shape[0] * mask.shape[1])
print(sum(sum(mask)) / (mask.shape[0] * mask.shape[1]))



tensor(1272.)
torch.Size([30, 2482])
74460
tensor(0.0171)


In [20]:
#Evaulating the model
model.eval()
with torch.no_grad():
    reconstructed = model(ratings_torch)

# Fill missing values in the original matrix
filled_data = ratings_torch.clone()
filled_data[mask_tensor == 0] = reconstructed[mask_tensor == 0]

print("Original Data:\n", ratings_torch)
print("Reconstructed Data:\n", reconstructed)
print("Filled Data:\n", filled_data)


Original Data:
 tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])
Reconstructed Data:
 tensor([[3.3440, 4.0150, 4.2053,  ..., 4.3530, 4.6351, 3.9124],
        [3.7271, 3.5444, 4.2222,  ..., 3.6607, 4.2960, 4.1987],
        [3.7271, 3.5444, 4.2222,  ..., 3.6607, 4.2960, 4.1987],
        ...,
        [4.2271, 2.8279, 4.4556,  ..., 4.2459, 4.5033, 4.2984],
        [3.7271, 3.5444, 4.2222,  ..., 3.6607, 4.2960, 4.1987],
        [3.7271, 3.5444, 4.2222,  ..., 3.6607, 4.2960, 4.1987]])
Filled Data:
 tensor([[3.3440, 4.0150, 4.2053,  ..., 4.3530, 4.6351, 3.9124],
        [3.7271, 3.5444, 4.2222,  ..., 3.6607, 4.2960, 4.1987],
        [3.7271, 3.5444, 4.2222,  ..., 3.6607, 4.2960, 4.1987],
        ...,
        [4.2271, 2.8279, 4.4556,  ..., 4.2459, 4.5033, 4.2984],
        [3.7271, 3.5444, 4.2

In [21]:
print(reconstructed[0].numpy())
print(ratings_torch[0].numpy())

print((reconstructed[0].numpy()-ratings_torch[0].numpy())/ratings_torch[0].numpy())

[3.343985  4.015009  4.2053337 ... 4.353     4.635142  3.912395 ]
[0. 0. 0. ... 0. 0. 0.]
[inf inf inf ... inf inf inf]


/tmp/ipykernel_1586/2166569457.py:4: RuntimeWarning: divide by zero encountered in divide
  print((reconstructed[0].numpy()-ratings_torch[0].numpy())/ratings_torch[0].numpy())


In [ ]:
pred_ratings_list = reconstructed[0].detach().numpy()

#give a list sorted out with books you've already read:
sorted_indices = np.argsort(pred_ratings_list)[::-1]
print("Top books are:")
list_num = 1
for idx in sorted_indices: 
#     print("ratings_matrix[user_id, idx]= ", ratings_matrix[user_id, idx])
    if  (ratings_matrix[user_id, idx] > 0) or(np.isnan(pred_ratings_list[idx])) :
        continue
    print("#", list_num , titles[idx], " - Predicted Rating:", round(pred_ratings_list[idx], 1))
    list_num += 1

Top books are:
# 1 The Complete Maus  - Predicted Rating: 5.0
# 2 Pippi Longstocking
        (Pippi LÃ¥ngstrump, #1)  - Predicted Rating: 5.0
# 3 The Wise Manâ€™s Fear
        (The Kingkiller Chronicle, #2)  - Predicted Rating: 5.0
# 4 A Dirty Job
        (Grim Reaper, #1)  - Predicted Rating: 5.0
# 5 The Book Thief  - Predicted Rating: 5.0
# 6 The Knife of Never Letting Go
        (Chaos Walking, #1)  - Predicted Rating: 5.0
# 7 Proven Guilty
        (The Dresden Files, #8)  - Predicted Rating: 5.0
# 8 Blood Rites
        (The Dresden Files, #6)  - Predicted Rating: 5.0
# 9 Summer Knight
        (The Dresden Files, #4)  - Predicted Rating: 5.0
# 10 Small Favor
        (The Dresden Files, #10)  - Predicted Rating: 5.0
# 11 Dead Beat
        (The Dresden Files, #7)  - Predicted Rating: 5.0
# 12 The Name of the Wind
        (The Kingkiller Chronicle, #1)  - Predicted Rating: 5.0
# 13 The Princess Diaries
        (The Princess Diaries, #1)  - Predicted Rating: 5.0
# 14 Changes
        (Th

# 210 Half Broke Horses  - Predicted Rating: 4.8
# 211 Home Front  - Predicted Rating: 4.8
# 212 Anthem  - Predicted Rating: 4.8
# 213 J.R.R. Tolkien 4-Book Boxed Set: The Hobbit and The Lord of the Rings  - Predicted Rating: 4.8
# 214 Blood of the Fold
        (Sword of Truth, #3)  - Predicted Rating: 4.8
# 215 Worth Dying For
        (Jack Reacher, #15)  - Predicted Rating: 4.8
# 216 The God of Small Things  - Predicted Rating: 4.8
# 217 Worst Case Scenario  - Predicted Rating: 4.8
# 218 The Hunger Games Trilogy Boxset
        (The Hunger Games, #1-3)  - Predicted Rating: 4.8
# 219 Before I Go to Sleep  - Predicted Rating: 4.8
# 220 Prodigy
        (Legend, #2)  - Predicted Rating: 4.8
# 221 Blindsighted
        (Grant County, #1)  - Predicted Rating: 4.8
# 222 Anne of Green Gables
        (Anne of Green Gables, #1)  - Predicted Rating: 4.8
# 223 Six of Crows
        (Six of Crows, #1)  - Predicted Rating: 4.8
# 224 Hunted
        (House of Night, #5)  - Predicted Rating: 4.8
# 225 B

In [ ]:
#making weighted loss matrix
percents = np.array([ 2.0839861,   6.38564535, 22.8939068,  37.94135873, 30.69510302])
each_weights = 100/percents
print(each_weights)
print(each_weights.sum())

print(each_weights * percents)

weights_array = np.zeros(ratings_torch.shape)
for i in tqdm(range(len(ratings_torch))):
    for j in range(len(ratings_torch[0])):
        for num in [1, 2, 3, 4, 5]:
            if ratings_torch[i,j] == num:
                weights_array[i,j] = each_weights[num-1]
weights_tensor = torch.tensor(weights_array)

In [ ]:
weights[0]

In [ ]:
#doing masked autoencoder with weighted loss
latent_dim = 100 # Number of latent features

model = SparseAutoencoder(num_items, latent_dim)
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Use MSE loss with weights but only consider observed values
def masked_mse_loss_diff(reconstructed, original, mask, weights):
    loss = (((reconstructed - original) ** 2) * mask)
    weighted_loss = loss * weights
    return weighted_loss.sum() / mask.sum()/100

#break up data into train and val
print("ratings_torch shape = ", ratings_torch.shape)
print("mask_tensor shape = ", mask_tensor.shape)
print("weights shape = ", weights_tensor.shape)

dataset = TensorDataset(ratings_torch, mask_tensor, weights_tensor) #keeping the mask
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)


#train the model
epochs = 5000
best_loss = 10e10
counter = 0
for epoch in range(epochs):
    train_loss = 0.0
    for inputs, mask, this_weight in train_loader:
        model.train()
        optimizer.zero_grad()

        # Forward pass
        reconstructed = model(inputs)
        loss = masked_mse_loss_diff(reconstructed, inputs, mask, this_weight)
#         loss_not_weighted = masked_mse_loss(reconstructed, inputs, mask)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()

    train_loss /= len(train_loader)
    
    if (epoch + 1) % 10 == 0:
#         print(f"Epoch {epoch + 1}/{epochs}, Loss: {train_loss}")
        #check validation
        model.eval()
        val_loss = 0.0
        val_loss_not_weighted = 0.0
        with torch.no_grad():
            for inputs, mask, this_weight in val_loader:
                outputs = model(inputs)
                loss = masked_mse_loss_diff(outputs, inputs, mask, this_weight)
                loss_not_weighted = masked_mse_loss(outputs, inputs, mask)
                val_loss += loss.item()
                val_loss_not_weighted += loss_not_weighted.item()
                

        val_loss /= len(val_loader)
        

        print(f"Epoch {epoch + 1} - Train Loss: {train_loss:.4f} - Val Loss: {val_loss:.4f} - Val not weighted: {val_loss_not_weighted:.4f}" )
        if val_loss < best_loss: #if improve then save
            #save model 
            torch.save(model.state_dict(), "model_weighted{}.pkl".format(latent_dim))
            best_loss = val_loss
            print("Model saved to model_weighted{}.pkl.".format(latent_dim))
            counter = 0
        else:
            counter += 10
            
    if counter > 200:
        print("Done training because of no improvement.")
        break
        
            


In [ ]:
dfghj
import torch
from sklearn.model_selection import KFold

# Mask for observed values (1 for observed, 0 for missing)
ratings_torch = torch.tensor(ratings).float()
mask = (ratings_torch != 0).float()
print(mask)


#Define autoencoder
class SparseAutoencoder(nn.Module):
    def __init__(self, num_items, latent_dim):
        super(SparseAutoencoder, self).__init__()
        self.encoder = nn.Linear(num_items, latent_dim)
        self.decoder = nn.Linear(latent_dim, num_items)
        
    def forward(self, x):
        encoded = torch.relu(self.encoder(x))
        decoded = self.decoder(encoded)
        # Scale sigmoid output to [1, 5]
        return 1 + 4 * torch.sigmoid(decoded)
        return decoded

    
#initialize the model
num_users, num_items = ratings_torch.shape
np.save("num_users.npy", np.array(num_users))
np.save("num_items.npy", np.array(num_items))

for latent_dim in [2, 5, 10, 20, 40, 50, 75, 100]:
    print("latent_dim = ", latent_dim)
# latent_dim = 20  # Number of latent features

    model = SparseAutoencoder(num_items, latent_dim)
    optimizer = optim.Adam(model.parameters(), lr=0.01)

    # Define your model, loss function, and optimizer
    # Assuming model, ratings_torch, mask, and optimizer are already defined

    epochs = 1000
    k_folds = 5  # Number of folds for cross-validation
    kf = KFold(n_splits=k_folds, shuffle=True)

    # Store the losses for each fold
    fold_losses = []

    for fold, (train_idx, val_idx) in enumerate(kf.split(ratings_torch)):
#         print(f"\nFold {fold + 1}/{k_folds}")

        # Split the data into training and validation sets
        train_ratings = ratings_torch[train_idx]
        val_ratings = ratings_torch[val_idx]
        train_mask = mask[train_idx]
        val_mask = mask[val_idx]

        # Initialize a new model for each fold
        model = SparseAutoencoder(num_items, latent_dim)
    #     optimizer = optim.Adam(model.parameters(), lr=0.01)

        # Re-initialize optimizer for each fold
        optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

        # Train the model
        for epoch in range(epochs):
            model.train()
            optimizer.zero_grad()

            # Forward pass for training
            reconstructed = model(train_ratings)
            loss = masked_mse_loss(reconstructed, train_ratings, train_mask)

            # Backward pass and optimization
            loss.backward()
            optimizer.step()

#             if (epoch + 1) % 1000 == 0:
#                 print(f"Epoch {epoch + 1}/{epochs}, Loss: {loss.item():.4f}")

        # Evaluate the model on the validation set
        model.eval()
        with torch.no_grad():
            reconstructed_val = model(val_ratings)
            val_loss = masked_mse_loss(reconstructed_val, val_ratings, val_mask)

        print(f"Validation Loss for Fold {fold + 1}: {val_loss.item():.4f}")

        # Store the validation loss for this fold
        fold_losses.append(val_loss.item())

    # Print the average validation loss after all folds
    print(f"\nAverage Validation Loss across all folds: {sum(fold_losses)/k_folds:.4f}")


In [ ]:
fgh

In [ ]:
ratings_matrix[user_id]

In [ ]:
# Convert the filled ratings data into a numpy array
ratings_matrix = ratings_df.values

# Initialize KNN (using user-based KNN)
import math
knn = NearestNeighbors(n_neighbors=math.ceil(num_users/10), metric='cosine')  # Using cosine similarity
knn.fit(ratings_matrix)

# Example: Predict rating for User 0 on Item 2
user_id = 0  # Index of user in the matrix
item_id = 9  # Index of item to predict rating for

# Get the nearest neighbors for user 0 (excluding the user itself)
distances, indices = knn.kneighbors([ratings_matrix[user_id]])

pred_ratings_list = np.array([])
rankings_list = np.array([])
for item_id in range(num_titles):
    # Get the ratings for the neighbors on item 2
    neighbor_ratings = np.array([ratings_matrix[i, item_id] for i in indices[0] if not np.isnan(ratings_matrix[i, item_id])])
    print(ratings_matrix[:, item_id])
    print(neighbor_ratings)
    ghjk
    
    predicted_rating = np.mean(neighbor_ratings[np.nonzero(neighbor_ratings)])
    rankings = np.sum(neighbor_ratings[np.nonzero(neighbor_ratings)])

#     print(predicted_rating)

    pred_ratings_list = np.append(pred_ratings_list, predicted_rating)
    rankings_list = np.append(rankings_list, rankings)
#     print(f"Predicted rating for User {user_id} on Item {item_id}: {predicted_rating}")
    
best_book_rating = np.max(pred_ratings_list)
best_book_idx = np.argmax(pred_ratings_list)

sorted_indices = np.argsort(pred_ratings_list)[::-1]
print("Top books are:")
for i, idx in enumerate(sorted_indices): 
    print("#", (i+1) , titles[idx], "Rating:", round(pred_ratings_list[idx], 1))

In [ ]:
import numpy as np
import scipy.sparse as sp
from scipy.sparse.linalg import svds

# Create a sparse matrix (CSR format)
A = np.array([[1.0, 0, 0], [0, 2, 3], [4, 0, 6], [0, 0, 0]])
# print(A)
sparse_matrix = sp.csr_matrix(ratings_matrix)

# Perform SVD on the sparse matrix
# k is the number of singular values to compute (you can choose a value smaller than min(m, n))
U, S, VT = svds(sparse_matrix, k=500)

# Output the matrices
print("U (Left Singular Vectors):\n", U)
print("\nS (Singular Values):\n", S)
print("\nVT (Right Singular Vectors - Transposed):\n", VT)

# Reconstruct the matrix from U, S, VT
S_full = np.diag(S)  # Convert singular values to a diagonal matrix
A_reconstructed = np.dot(U, np.dot(S_full, VT))

print("\nReconstructed Matrix A:\n", A_reconstructed)


In [ ]:
my_diff = (ratings_matrix[0]- A_reconstructed[0])
print(ratings_matrix.shape)
plt.plot(my_diff, '.')

# for i in range(len(ratings_matrix[0])):
#     if ratings_matrix[0, i] > 0:
#         print(ratings_matrix[0, i], A_reconstructed[0, i], titles[i])
        
for i in range(len(ratings_matrix[0])):
    if ratings_matrix[0, i] == 0 and A_reconstructed[0, i] > 0:
        print(ratings_matrix[0, i], A_reconstructed[0, i], titles[i])


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Create a custom colormap with white for 0 and red for non-zero
cmap = mcolors.ListedColormap(['white', 'red'])
bounds = [0, 0.1, 1]  # Set bounds for 0 (white) and non-zero (red)
norm = mcolors.BoundaryNorm(bounds, cmap.N)

# plt.imshow(ratings_matrix - A_reconstructed, cmap=cmap, norm=norm)
plt.plot(ratings_matrix - A_reconstructed)
plt.show()

In [ ]:
ratings_matrix - A_reconstructed

In [ ]:
from sklearn.cluster import DBSCAN
dbscan = DBSCAN(eps=0.75, min_samples=2, metric='cosine')
labels = dbscan.fit_predict(ratings_matrix)


In [ ]:
print(list(set(labels)))
print(labels.shape)

In [ ]:
idx_in_group = np.arange(len(labels))
filtered_users = [idx_in_group for idx_in_group, label in zip(idx_in_group, labels) if label == 0]
print(filtered_users)

In [ ]:
from sklearn.cluster import SpectralClustering
from scipy.sparse import csr_matrix
import numpy as np

# Example sparse data
# X = np.random.rand(100, 2)
X_sparse = csr_matrix(ratings_matrix)

n_clusters = 50
# Apply Spectral Clustering
spectral = SpectralClustering(n_clusters=n_clusters, affinity='nearest_neighbors')
labels = spectral.fit_predict(X_sparse)

print(labels)
print(list(set(labels)))
print(labels.shape)

In [ ]:
# idx_in_group = np.arange(len(labels))
# filtered_users = [idx_in_group for idx_in_group, label in zip(idx_in_group, labels) if label == 0]
# print(filtered_users)

group_averages = []

for group in range(n_clusters):
    # Find indices of users in the current group
    group_users = np.where(labels == group)[0]
    
    # Extract the rows for users in this group
    group_data = ratings_matrix[group_users]
    
    print("Number of perople in group = ", group_data.shape[0])
    
    pred_ratings_list = np.array([])
    for item_id in range(num_titles):
        data = group_data[:,item_id]
        predicted_rating = np.mean(data[np.nonzero(data)])
#                 rankings = np.sum(neighbor_ratings[np.nonzero(neighbor_ratings)])

        pred_ratings_list = np.append(pred_ratings_list, predicted_rating)
    # Compute the average for each column (item) for this group
#     group_avg = np.mean(group_data[np.nonzero(group_data)], axis=0)
    
    # Append the average for this group
    group_averages.append(pred_ratings_list)

# Convert the list of group averages to a numpy array for easy viewing
group_averages = np.array(group_averages)

# Display the average for each item in each group
print("Average preferences for each item by group:")
print(group_averages.shape)

In [ ]:
group = labels[0]
print("my group = ", group)
sorted_indices = np.argsort(group_averages[group])[::-1]
print(sorted_indices)
for i in sorted_indices:
    if (ratings_matrix[0, i] > 0) or (np.isnan(group_averages[group, i])):
        pass
    else:
        print(titles[i], round(group_averages[group,i], 1))